In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline
import pandas as pd
from nltk.corpus import stopwords

In [ ]:
dados = pd.read_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/MaskedLanguageModeling/lacunaUnicaIrb.csv')
dados

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
indice = 7
dados['text'][indice]

In [ ]:
BERTimbauModel = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')
BERTimbauTokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [ ]:
fill = pipeline('fill-mask', model=BERTimbauModel, tokenizer=BERTimbauTokenizer)

In [ ]:
fill(indice)

In [ ]:
HelBERTModel = AutoModelForMaskedLM.from_pretrained('/var/projetos/Jupyterhubstorage/victor.silva/brazilian-legal-text-bert/Modelos/epocas_HelBERT-base-uncased/checkpoint-epoca-30')
HelBERTTokenizer = AutoTokenizer.from_pretrained('/var/projetos/Jupyterhubstorage/victor.silva/brazilian-legal-text-bert/Modelos/epocas_HelBERT-base-uncased/checkpoint-epoca-30')

In [ ]:
fill = pipeline('fill-mask', model=HelBERTModel, tokenizer=HelBERTTokenizer)

In [ ]:
fill(indice)

### Buscando no dataset palavras do Tesauro IRB

In [ ]:
import json
import pandas as pd
from unidecode import unidecode
from tqdm.auto import tqdm
import re

In [ ]:
with open('/var/projetos/Jupyterhubstorage/victor.silva/brazilian-legal-text-bert/Datasets/HelBERT-base-uncased-scratch/teste.txt', 'r') as f:
    dados = [line.strip() for line in f.readlines()]
dados = pd.DataFrame(dados, columns=['text'])
dados

In [ ]:
tesauro = json.load(open('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/MaskedLanguageModeling/tesauroIRB3Palavras.json'))
tesauro

In [ ]:
lista_palavras = []
for conj in tesauro:
    lista_palavras.append(unidecode(conj['Palavra']).lower())

In [ ]:
lista_palavras = list(set(lista_palavras))

In [ ]:
sentencas = []
dados['label'] = ''
for index in tqdm(dados.index):
    for palavra in lista_palavras:
        if re.search(r'\b' + palavra + r'\b', dados['text'][index]):
            dados['text'][index] = re.sub(r'\b' + palavra + r'\b', f'[MASK]', dados['text'][index])
            dados['label'][index] = palavra
            sentencas.append(dados.iloc[index])
            break

In [ ]:
dados = dados[dados['label'] != '']
dados

In [ ]:
dados.drop_duplicates(subset=['label'], inplace=True)
dados.reset_index(drop=True, inplace=True)
dados

In [ ]:
dados.to_csv('/var/projetos/Jupyterhubstorage/victor.silva/HelBERTModel/Datasets/MaskedLanguageModeling/lacunaUnicaIrb.csv', index=False)

### Testando MLM

In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline

In [ ]:
modelo = AutoModelForMaskedLM.from_pretrained('neuralmind/bert-base-portuguese-cased')
tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-base-portuguese-cased')

In [ ]:
frase = 'Tem uma [MASK] no meio do caminho.'

In [ ]:
labels = ['pedra']

In [ ]:
fill = pipeline('fill-mask', model=modelo, tokenizer=tokenizer)

In [ ]:
fill(frase)